In [1]:
pip install selenium pandas webdriver-manager


  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/b5/b5/3bd0b038d80950ec13e6a2c8d03ed8354867dc60064b172f2f4ffac8afbe/webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager

# Excel dosyasını yükleyelim

df = pd.read_excel("amazon matching.xlsx")

# ChromeDriver kurulumunu başlatalım
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Amazon'a erişim için URL
amazon_url = "https://www.amazon.ae"

# Ürün bağlantılarını saklayacak yeni bir sütun oluşturalım
df["Amazon_Link"] = None

# Her bir başlık için arama yapacağız
for index, row in df.iterrows():
    product_title = row['Title']
    
    # Amazon ana sayfasına git
    driver.get(amazon_url)
    
    # Arama çubuğunu bul ve başlığı ara
    search_box = driver.find_element(By.ID, "twotabsearchtextbox")
    search_box.clear()
    search_box.send_keys(product_title)
    search_box.send_keys(Keys.RETURN)
    
    # Sayfanın yüklenmesi için kısa bir bekleme
    time.sleep(2)
    
    try:
        # İlk sonucu bul ve bağlantıyı al
        first_result = driver.find_element(By.CSS_SELECTOR, ".s-main-slot .s-result-item h2 a")
        product_link = first_result.get_attribute("href")
        df.at[index, "Amazon_Link"] = product_link  # Linki DataFrame'e ekleyelim
    except Exception as e:
        print(f"Ürün bulunamadı: {product_title}")
        df.at[index, "Amazon_Link"] = "Link bulunamadı"
    
    # Bir süre bekleyerek çok hızlı istek yapılmasını engelleyelim
    time.sleep(1)

# Tarayıcıyı kapatalım
driver.quit()

# Sonucu yeni dosyaya kaydedelim
output_file_path = "amazon_matching_with_links.xlsx"  # Kayıt yeri olarak betiğin olduğu dizini kullanır
df.to_excel(output_file_path, index=False)

print(f"Tamamlandı! Dosya kaydedildi: {output_file_path}")
